In [1]:
#!/usr/bin/env python3

import chardet
import csv
import json
import nibabel as nib
import nilearn
import numpy as np
import os
import pandas as pd
import random
import regex as re
import shutil
import string
import struct
import sys

from chardet import UniversalDetector as udet
from collections import Counter
from functools import reduce
from io import StringIO
from io import BytesIO
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os import listdir as ls
from os.path import join as pjoin
from pandas import DataFrame as df
from string import printable
from typing import Union
from zipfile import ZipFile
from tqdm import tqdm
from typing import Generator
# from typing import list_iterator
from unidecode import unidecode
from operator import itemgetter

from nilearn.glm.first_level import make_first_level_design_matrix as mfldm
from nilearn.glm.first_level import first_level_from_bids
from nilearn.plotting import plot_design_matrix as plot1dm

from sniffbytes import flatten
from sniffbytes import loadfiles
from sniffbytes import loadimages

from bidsify_utils import bidsify_load_scans
from bidsify_utils import bidsify_names

# from json_read import json_read

import sniffbytes as snif

from scanzip import scanzip
# from fetch_cimaq import fetch_cimaq


from removeEmptyFolders import removeEmptyFolders
# from multiple_replace import multiple_replace

from cimaq import cimaq

/home/fnadeau/.local/lib/python3.6/site-packages/nilearn-0.7.0-py3.6.egg/nilearn/glm/__init__.py:56: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  'It may change in any future release of Nilearn.', FutureWarning)


In [3]:
cimaq_dir = xpu('~/../../data/simexp/datasets/cimaq_03-19/')
# from fetch_cimaq import cimaq
cimaq = cimaq.fetch(cimaq_dir)

scanning archive: 100%|██████████| 94/94 [00:01<00:00, 47.18it/s]
indexing participants: 282it [00:00, 594.56it/s]
cleaning: 282it [00:05, 50.83it/s]
loading subjects: 100%|██████████| 94/94 [00:00<00:00, 413.90it/s]


In [4]:
cimaq

,subid,events,behav,subid,subid,events
0,3025432_658178,trialnumber trial_type trialcode oldnumber...,category stim old...,motion_tx motion_ty motion_tz motion_r...,108391,filena...
1,3123186_920577,trialnumber trial_type trialcode oldnumber...,category stim oldnum...,motion_tx motion_ty motion_tz motion_r...,120839,filena...
2,3163875_199801,trialnumber trial_type trialcode oldnumber...,category stim oldnum...,motion_tx motion_ty motion_tz motion_r...,122922,filen...
3,3291977_748676,trialnumber trial_type trialcode oldnumber...,category stim o...,motion_tx motion_ty motion_tz motion_r...,127228,filen...
4,3400972_956130,trialnumber trial_type trialcode oldnumber...,category stim ...,motion_tx motion_ty motion_tz motion_r...,139593,filen...
...,...,...,...,...,...,...
89,9003010_785245,trialnumber trial_type trialcode oldnumber...,category stim ol...,motion_tx motion_ty motion_tz motion_r...,979001,filename...
90,9249304_778749,trialnumber trial_type trialcode oldnumber...,category stim old...,motion_tx motion_ty motion_tz motion_r...,983291,filename...
91,9296157_955548,trialnumber trial_type trialcode oldnumber...,category stim ...,motion_tx motion_ty motion_tz motion_r...,988602,file...
92,9494076_630120,trialnumber trial_type trialcode oldnumber...,category stim o...,motion_tx motion_ty motion_tz motion_r...,996599,filename...


In [5]:
cimaq_dir = xpu('~/../../data/simexp/datasets/cimaq_03-19/')


class CIMAQ:
    @ classmethod
    def fetch(self, cimaq_dir: Union[str, os.PathLike]):
        return fetch_cimaq(cimaq_dir)

cimaq = CIMAQ.fetch(cimaq_dir)

scanning archive: 100%|██████████| 94/94 [00:01<00:00, 50.67it/s]
indexing participants: 282it [00:00, 598.46it/s]
cleaning: 282it [00:05, 51.55it/s]
loading subjects: 100%|██████████| 94/94 [00:00<00:00, 405.41it/s]


In [ ]:
cimaq

### Nilearn First Level Matrix parameters explained:
- https://nilearn.github.io/modules/generated/nilearn.glm.first_level.make_first_level_design_matrix.html#nilearn.glm.first_level.make_first_level_design_matrix


#### The pd.DataFrame instance might have these keys:
##### "onset" and "duration" are mandatory

- ‘onset’: column to specify the start time of each events in
   seconds. An error is raised if this key is missing.

- ‘trial_type’: column to specify per-event experimental conditions
   identifier. If missing each event are labelled ‘dummy’ and considered to form a unique condition.

- ‘duration’: column to specify the duration of each events in
   seconds. If missing the duration of each events is set to zero.

- ‘modulation’: column to specify the amplitude of each
   events. If missing the default is set to ones(n_events).



### Why Encoding Type Matters

- See how encoding type formats the same "null-byte value" differently \
  when printed to be human readable

In [ ]:
display("\x00".encode('utf8'), "\x00".encode('utf16'), "\x00".encode('utf32'),
        "".encode('utf8'), "".encode('utf16'), "".encode('utf32'))

In [ ]:
from collections import namedtuple
help(namedtuple)

In [ ]:
import os
import pandas as pd
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os import listdir as ls

from typing import Union
from pandas import DataFrame as df
import sniffbytes as sniff
from bidsify_utils import bidsify_load_scans
from bidsify_utils import bidsify_names

def xtrct_cimaq(cimaq_dir: Union[str, os.PathLike]) -> np.ndarray:
    return df(tuple(scanzip(apath,
                            exclude = ['Practice', 'Pratique',
                                       'PRATIQUE', 'PRACTICE', 'READ',
                                       'Encoding-scan', 'Retrieval-'],
                            to_xtrct = ['.pdf', '.edat2'],
                            dst_path = pjoin(os.getcwd(), 'newdevs',
                                             'cimaq_uzeprimes'))
                    for apath in
                    tqdm(snif.filter_lst_inc(snif.clean_bytes(
                        xpu(pjoin(cimaq_dir, 'derivatives/CIMAQ_fmri_memory/data',
                                  'participants/sub_list_TaskQC.tsv'))).decode(
                        ).split()[1:], snif.loadimages(xpu(pjoin(
                        cimaq_dir, 'derivatives/CIMAQ_fmri_memory/data',
                        'task_files/zipped_eprime')))))))[0].values.flatten()

def fix_cimaq(cimaq_dir: Union[str, os.PathLike]) -> None:
    os.makedirs(pjoin(os.getcwd(), 'newtest', 'events'), exist_ok = True)
    os.makedirs(pjoin(os.getcwd(), 'newtest', 'behavioural'), exist_ok = True)
    os.makedirs(pjoin(os.getcwd(), 'newdevs', 'cimaq_uzeprimes'), exist_ok = True)
    for val in tqdm(xtrct_cimaq(cimaq_dir), 'fixing cimaq'):
        # creating events files
        (pd.concat([snif.bytes2df(val['bsheets'].values[1],
                                  has_header = None),
                    snif.bytes2df(val['bsheets'].values[0]).loc[:snif.bytes2df(
                        val['bsheets'].values[1],
                        has_header = None).shape[0] -1, :].drop(
                        columns = [0,1,2,3, 4, 6],
                        index = 0).rename(columns = {5: 'onset',
                                                     7: 'fix_onset',
                                                     8: 'fix_duration'})],
                   axis = 1).to_csv(pjoin(os.getcwd(), 'newtest', 'events', 'sub-_' + \
                                          '_'.join(val['filename'].values[0].replace('-', '_').split(
            '_')[:2])+'_run-01_task-encoding_events.tsv'), sep = '\t', index = None),
         # creating behavioural files
         snif.bytes2df(val['bsheets'].values[2], has_header = True).to_csv(pjoin(
             os.getcwd(), 'newtest', 'behavioural', 'sub-_' + '_'.join(
                 val['filename'].values[0].replace('-', '_').split('_')[:2]) + \
                 '_run-01_task-encoding_behavioural.tsv'), sep = '\t', index = None))

def cimaq2nilearn(cimaq_dir: Union[str, os.PathLike]) -> None:
    fix_cimaq(cimaq_dir)
    events = snif.loadfiles(snif.loadimages(pjoin(
                           os.getcwd(), 'newtest', 'events')))
    os.makedirs(pjoin(os.getcwd(), 'newtest', 'nilearn_events'), exist_ok = True)
    for row in tqdm(events.iterrows(), desc = 'conforming events.tsv files'):
        sheet = pd.read_csv(row[1].fpaths, sep = '\t')
        
        sheet['duration'] = sheet['onset'] + \
                                    sheet['fix_duration'] + \
                                    sheet['onset'].sub(
                                    sheet['fix_onset']).round(0)
        sheet.rename(columns = {'category': 'trial_type'}).set_index(
            'trialnumber').to_csv(pjoin(os.getcwd(), 'newtest',
                                        'nilearn_events', row[1].filename + '.tsv'),
                                  sep = '\t', index = None)

def fetch_cimaq(cimaq_dir: Union[str, os.PathLike]) -> dict:
    cimaq2nilearn(cimaq_dir)    
    scan_infos = bidsify_load_scans(cimaq_dir, snif.clean_bytes(xpu(pjoin(
            cimaq_dir, 'derivatives/CIMAQ_fmri_memory/data',
            'participants/sub_list_TaskQC.tsv'))).decode().split()[1:])

    scan_infos['dccid'] = sorted([(filename, filename.split('-')[1].split('_')[0])[1]
                                  for filename in scan_infos.filename])
    behav = snif.loadfiles(snif.loadimages(pjoin(
                           os.getcwd(), 'newtest', 'behavioural')))
    behav[['pscid', 'dccid']] = [filename.split('_')[1:3]
                                  for filename in behav.filename]
    events = snif.loadfiles(snif.loadimages(pjoin(
                           os.getcwd(), 'newtest', 'nilearn_events')))
    events[['pscid', 'dccid']] = [filename.split('_')[1:3]
                                  for filename in events.filename]

    confounds = snif.loadfiles(snif.filter_lst_inc(
        snif.clean_bytes(xpu(pjoin(cimaq_dir, 'derivatives/CIMAQ_fmri_memory/data',
                                   'participants/sub_list_TaskQC.tsv'))).decode().split()[1:],
        snif.loadimages( pjoin(cimaq_dir, 'derivatives/CIMAQ_fmri_memory/data',
                                  'confounds', 'resample'))))
    confounds['bids_names'] = [bidsify_names(filename) for
                               filename in confounds.filename]
    
    subs = df(((grp, scan_infos.groupby('dccid').get_group(grp))
               for grp in tqdm(scan_infos.groupby('dccid').groups,
                               desc = 'loading subjects')),
              columns = ['subject', 'scans']).set_index(
                            'subject').sort_index().reset_index(
                         drop = False)
    return dict(zip(['scans', 'behavior', 'confounds', 'events'],
                    [subs, behav, confounds, events]))

class CIMAQ:
    def __init__(self, cimaq_dir: Union[str, os.PathLike]) -> None:
        self.cimaq_dir = cimaq_dir
    @ classmethod
    def fetch(self, cimaq_dir: Union[str, os.PathLike]):
        return fetch_cimaq(cimaq_dir)

cimaq = CIMAQ.fetch(cimaq_dir)

In [ ]:
# cimaq.fetch(cimaq_dir)
cimaq

In [ ]:
cimaq

In [ ]:
# scan_infos = bidsify_load_scans(cimaq_dir, snif.clean_bytes(xpu(pjoin(
#             drv_dir, 'participants/sub_list_TaskQC.tsv'))).decode().split()[1:])

# scan_infos['dccid'] = sorted([(filename, filename.split('-')[1].split('_')[0])[1]
#                             for filename in scan_infos.filename])
# behav = snif.loadfiles(snif.loadimages(pjoin(
#                        os.getcwd(), 'newtest', 'behavioural')))
# behav[['pscid', 'dccid']] = [filename.split('_')[1:3]
#                               for filename in behav.filename]
# events = snif.loadfiles(snif.loadimages(pjoin(
#                        os.getcwd(), 'newtest', 'events')))
# events[['pscid', 'dccid']] = [filename.split('_')[1:3]
#                               for filename in events.filename]

# confounds = snif.loadfiles(snif.loadimages(pjoin(drv_dir, 'confounds', 'resample')))
# confounds['bids_names'] = [bidsify_names(filename) for
#                            filename in confounds.filename]

# # confounds[['pscid', 'dccid']] = [filename.split('_')[1:3]
# #                                  for filename in confounds.filename]


In [ ]:


subs = df(((grp, scan_infos.groupby('dccid').get_group(grp))
           for grp in scan_infos.groupby('dccid').groups),
          columns = ['subject', 'scans']).set_index(
                        'subject').sort_index().reset_index(
                     drop = False)


In [ ]:
display(events, behav, subs)
# subs = df(((grp, scan_infos.groupby('filename').get_group(grp))
#            for grp in scan_infos.groupby('filename').groups),
#           columns = ['subject', 'scans']).set_index(
#                         'subject').sort_index().reset_index(
#                      drop = False)




# jsonfiles = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                               if row[1].ext == '.json']]
# scan_infos = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                              if row[0] not in jsonfiles.index]]


In [ ]:
subs.iloc[44].scans

In [ ]:
byfilename = df((df(((grp, row[1].scans.groupby('filename').get_group(grp))
                 for grp in row[1].scans.groupby('filename').groups),
                columns = ['modality', 'scans']) for row in subs.iterrows())).values.flat
# json.dumps([itm.values.tolist() for itm in byfilename.values.tolist()])

In [ ]:
scan_infos.modality.unique()

list(next((itm))['scans'].values.tolist() for itm in byfilename)

In [ ]:
scan_infos

# By subject
# subs = df(((grp, scan_infos.groupby('dccid').get_group(grp))
#            for grp in scan_infos.groupby('dccid').groups),
#           columns = ['subject', 'scans']).set_index(
#                         'subject').sort_index().reset_index(
#                      drop = False)

files_per_sub = pd.Series(row[1].scans.shape[0] for row
                          in subs.iterrows()).describe()
files_per_sub
# By scan type (parent column)
# mods = df(((grp, scan_infos.groupby('parent').get_group(grp))
#            for grp in scan_infos.groupby('parent').groups),
#           columns = ['scan_type', 'scans']).set_index('scan_type')


# display(mods.loc['dwi']['scans'],
#         mods.loc['anat']['scans'])
# dwi_scans = [mods['dwi']] + [dwi_only.groupby('ext').get_group(grp)
#                              for grp in dwi_only.groupby('ext').groups]
# nifti_only = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                              if '.nii' in row[1].ext]]

# dwi_only = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                            if '.nii' not in row[1].ext
#                            and '.json' not in row[1].ext]]


In [ ]:
jsonfiles
from io import StringIO

def json_bytes(apath: Union[str, os.PathLike]):
    with open(apath, 'rb', buffering = 0) as jfile:
        jbytes3 = json.load(jfile)
    jfile.close()
    return jbytes3

# jsons = list((row[1].filename, json_bytes(row[1].fpaths))
#              for row in tqdm(jsonfiles.iterrows()))


In [ ]:
# list(pd.read_csv(apath, sep = '\t') for apath in events.fpaths)
# display(behav, events, scan_infos, jsonfiles, nifti_only, dwi_only)

# nifti_only['jsoninfo'] = jsonfiles.fpaths.tolist()

# mods = dict((grp, nifti_only.groupby('parent').get_group(grp))
#         for grp in nifti_only.groupby('parent').groups)





# nifti_only[['scans', 'parameters']] = tuple(((nib.load(fpath),
#                                               nib.load(fpath).header)
#                                              for fpath in nifti_only.fpaths))
# full_infos = pd.concat([df.from_dict({**dict(nib.load(row[1].fpaths).header),
#                                       **json_bytes(row[1].jsoninfo)},
#                                      orient = 'index').T
#                         for row in tqdm(nifti_only.iterrows())], join = 'inner')
# full_infos['filename'] = nifti_only['filename'].tolist()
# full_infos = full_infos.set_index('filename')
# full_infos.T

In [ ]:
display(dwi_scans[0], dwi_scans[1], dwi_scans[2])

In [ ]:
df.from_dict(mods['dwi'], orient ='index').T#, dwi_only

In [ ]:
full_infos.to_csv(pjoin(os.getcwd(), 'newdevs',
                        'inner_join_nifti_header_json.tsv'),
                  sep = '\t', index = None)

test = [row[1].values for row in full_infos.T.iterrows()]

test
# nifti_only[['scans', 'parameters']] = tuple(((nib.load(fpath),
#                                               nib.load(fpath).header)
#                                              for fpath in nifti_only.fpaths))

In [ ]:
# Ungrouped

from nilearn import image
from dict2csv import dict2csv

# jsonfiles = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                               if row[1].ext == '.json']]
# scan_infos = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                              if row[0] not in jsonfiles.index]]
# nifti_only = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                              if '.nii' in row[1].ext]]


# dwi_only = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                            if '.nii' not in row[1].ext
#                            and '.json' not in row[1].ext]]

# nifti_only = nifti_only.loc[[row[0] for row in nifti_only.iterrows()
#                              if row[1].fpaths not in dwi_only.fpaths]]





In [ ]:
test = [itm[1].unique()[0] for itm in
 df((itm[1].columns for itm
     in jsons)).dropna().iteritems()
 if len(itm[1].unique()) == 1]

In [ ]:
test

In [ ]:
json.dumps(unidecode(jsons[0][1][1].decode())).encode().split(
    snif.get_lineterminator(json.dumps(str(
        jsons[0][1][1].decode())).encode()))[0]

In [ ]:
os.makedirs(pjoin(os.getcwd(), 'cimaq_json_headers'), exist_ok = True)

cimaq_json_parameters = list((df.from_dict(dict(row[1].parameters),
                                          orient = 'index').to_csv(
                            pjoin(os.getcwd(), 'cimaq_nifti_headers', row[1].filename + \
                                  '_nifti_headers.tsv'), sep = '\t', index = None))
                             for row in nifti_only.iterrows())
cimaq_scan_parameters

In [ ]:
os.makedirs(pjoin(os.getcwd(), 'cimaq_nifti_headers'), exist_ok = True)

cimaq_scan_parameters = list((df.from_dict(dict(row[1].parameters),
                                          orient = 'index').to_csv(
                            pjoin(os.getcwd(), 'cimaq_nifti_headers', row[1].filename + \
                                  '_nifti_headers.tsv'), sep = '\t'))
                             for row in nifti_only.iterrows())
cimaq_scan_parameters

In [ ]:
import json
def dict2json(adict, dst_path: Union[str, os.PathLike]):

#     adict = {
#       'bill': 'tech',
#       'federer': 'tennis',
#       'ronaldo': 'football',
#       'woods': 'golf',
#       'ali': 'boxing'
#     }

    with open(dst_path, 'w') as json_file:
        json.dump(adict, json_file)
        json_file.close()

dict2json(cimaq_scan_parameters, pjoin(os.getcwd(), 'cimaq_scan_parameters.txt'))

In [ ]:
pjoin(os.getcwd(), 'cimaq_scan_parameters.csv')
cimaq_scans_json = json.load(pjoin(os.getcwd(), 'cimaq_scan_parameters.json'))
cimaq_scans_json

In [ ]:

dict2csv(cimaq_scan_parameters, pjoin(os.getcwd(), 'cimaq_scan_parameters.txt'))

list(cimaq_scan_parameters.keys())
# with open(pjoin(os.getcwd(), 'cimaq_scan_parameters.json'), 'w') as jfile:
#     json.dump(cimaq_scan_parameters, pjoin(os.getcwd(), 'cimaq_scan_parameters.json'))jfile.write(cimaq_scan_parameters)
# #     jfile.close()

# cimaq_scan_parameters

In [ ]:
params = pd.concat([df.from_dict(dict(row[1].parameters), orient = 'index')
                   for row in nifti_only.iterrows()], axis = 1).T
# params['filename'] = nifti_only.filename
# infos = df(df(((pd.Series(list(itm[1].values for itm in nifti_only.iteritems()) + \
#         list(itm[1].values for itm in params.iteritems()))))).T.values)
display(infos)
# nifti_only[params.columns] = [params.iteritems()]
# nifti_only

# pd.merge(params, nifti_only, on = 'filename', how = 'outer', axis =1)
# pd.concat((nifti_only, params), axis = 1)

In [ ]:
# Plotting anatomical images

from nilearn import plotting

nif = nifti_only.iloc[:5]
display([(row[1].filename,
          plotting.plot_img(image.load_img(row[1]['scans'])))
         for row in tqdm(nif.iterrows(), 'Plotting anatomical images')
 if 'anat' in row[1]['fpaths']])

In [ ]:


# # All ungrouped
# from nilearn import image

# jsonfiles = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                               if row[1].ext == '.json']]
# scan_infos = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                              if row[0] not in jsonfiles.index]]
# nifti_only = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                              if '.nii' in row[1].ext]]


# dwi_only = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                            if '.nii' not in row[1].ext
#                            and '.json' not in row[1].ext]]

# nifti_only = nifti_only.loc[[row[0] for row in nifti_only.iterrows()
#                              if row[1].fpaths not in dwi_only.fpaths]]


# nifti_only[['scans', 'parameters']] = tuple(((nib.load(fpath),
#                                               nib.load(fpath).header)
#                                              for fpath in nifti_only.fpaths))

# nif = nifti_only.iloc[:5]
# from nilearn import plotting
# display([(row[1].filename,
#           plotting.plot_img(image.load_img(row[1]['scans'])))
#          for row in nif.iterrows()
#  if 'anat' in row[1]['fpaths']])

In [ ]:
# # All ungrouped
# from nilearn import image

# jsonfiles = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                               if row[1].ext == '.json']]
# scan_infos = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                              if row[0] not in jsonfiles.index]]
# nifti_only = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                              if '.nii' in row[1].ext]]


# dwi_only = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                            if '.nii' not in row[1].ext
#                            and '.json' not in row[1].ext]]

# nifti_only = nifti_only.loc[[row[0] for row in nifti_only.iterrows()
#                              if row[1].fpaths not in dwi_only.fpaths]]


# nifti_only[['scans', 'parameters']] = tuple(((nib.load(fpath),
#                                               nib.load(fpath).header)
#                                              for fpath in nifti_only.fpaths))

# # All ungrouped
# from nilearn import image

# jsonfiles = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                               if row[1].ext == '.json']]
# scan_infos = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                              if row[0] not in jsonfiles.index]]
# nifti_only = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                              if '.nii' in row[1].ext]]


# dwi_only = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                            if '.nii' not in row[1].ext
#                            and '.json' not in row[1].ext]]

# nifti_only = nifti_only.loc[[row[0] for row in nifti_only.iterrows()
#                              if row[1].fpaths not in dwi_only.fpaths]]


# nifti_only[['scans', 'parameters']] = tuple(((nib.load(fpath),
#                                               nib.load(fpath).header)
#                                              for fpath in nifti_only.fpaths))

# nif = nifti_only.iloc[:5]
# from nilearn import plotting
# display([(row[1].filename,
#           plotting.plot_img(image.load_img(row[1]['scans'])))
#          for row in nif.iterrows()
#  if 'anat' in row[1]['fpaths']])
# # test = df(zip(nifti_only.iterrows(), jsonfiles.iterrows()))

# # test = [behav, events, scan_infos, jsonfiles, nifti_only, dwi_only]
# # test = [list(((itm.groupby('dccid').get_group(grp).groupby('modality').get_group(mod)
# #               for grp in itm.groupby('dccid').groups) for mod in
# #              itm.groupby('dccid').get_group(grp).groupby('modality').groups))
# #         for itm in test]


# # jsonfiles, scan_infos= (scan_infos.drop([[row[0]
# #                                 for row in scan_infos.iterrows()
# #                                  if row[1].ext != '.json']][0], axis=0),
# #             scan_infos.drop([[row[0]
# #                                 for row in scan_infos.iterrows()
# #                                  if row[1].ext == '.json']][0], axis=0))
# # display(behav, events, scan_infos, jsonfiles, nifti_only, dwi_only, test)
# image.load_img(nifti_only.iloc[0]['scans'])
# # display(test.iloc[0][0].scans)

In [ ]:
# # All ungrouped
# from nilearn import image

# jsonfiles = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                               if row[1].ext == '.json']]
# scan_infos = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                              if row[0] not in jsonfiles.index]]
# nifti_only = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                              if '.nii' in row[1].ext]]


# dwi_only = scan_infos.loc[[row[0] for row in scan_infos.iterrows()
#                            if '.nii' not in row[1].ext
#                            and '.json' not in row[1].ext]]

# nifti_only = nifti_only.loc[[row[0] for row in nifti_only.iterrows()
#                              if row[1].fpaths not in dwi_only.fpaths]]


# nifti_only[['scans', 'parameters']] = tuple(((nib.load(fpath),
#                                               nib.load(fpath).header)
#                                              for fpath in nifti_only.fpaths))

# nif = nifti_only.iloc[:5]
# from nilearn import plotting
# display([(row[1].filename,
#           plotting.plot_img(image.load_img(row[1]['scans'])))
#          for row in nif.iterrows()
#  if 'anat' in row[1]['fpaths']])

In [ ]:
# nif = nifti_only.iloc[:5]
# from nilearn import plotting
# display([(row[1].filename,
#           plotting.plot_img(image.load_img(row[1]['scans'])))
#          for row in nif.iterrows()
#  if 'anat' in row[1]['fpaths']])

In [ ]:
# epis = nifti_only.iloc[14]

In [ ]:
# ############ TODO: Group by modality ################3333

# subs_json = [jsonfiles.groupby('dccid').get_group(grp)
#              for grp in jsonfiles.groupby('dccid').groups]
# modgrps = [[sjson.groupby('modality').get_group(grp) for grp in sjson.groupby('modality').groups]
#            for sjson in subs_json]
# modgrps[4][0]
# # loaded_json = [pd.read_json(row[1].fpaths) for row in itm.iterrows()]
# #                for itm in subs_json]

# # loaded_json2 = [row[1].unique() for row in loaded_json.iterrows()]
# # loaded_json2 = [Counter(row[1]).most_common(1) for row in loaded_json.iterrows()]
# # loaded_json2

In [ ]:
# def _update_timestamp(path: os.PathLike, set_new: bool) -> None:
#     """
#     Context manager to set the timestamp of the path to plus or
#     minus a fixed delta, regardless of modifications within the context.

#     if set_new is True, the delta is added. Otherwise, the delta is subtracted.
#     """
#     stats = os.stat(path)
#     if set_new:
#         new_timestamp = (stats.st_atime_ns + _TIMESTAMP_DELTA, stats.st_mtime_ns + _TIMESTAMP_DELTA)
#     else:
#         new_timestamp = (stats.st_atime_ns - _TIMESTAMP_DELTA, stats.st_mtime_ns - _TIMESTAMP_DELTA)
#     try:
#         yield
#     finally:
#         os.utime(path, ns=new_timestamp)


# # Public Methods 